In [1]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy
from spacy.language import Language

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

# remove non english reviews
from langdetect import detect

/Users/de11675/.conda/envs/jupyterLab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# upload the dataset 
df = pd.read_excel('audi_reviews_eng_adjusted.xlsx')
df

,Username,Description,Number of reviews,Profile URL,Image URL,Review,Branch,Rating,Date,Translated,Year,Profile picture
0,Hayrullah AYAN,NaN,2,https://www.google.com/maps/contrib/1009337064...,https://lh3.googleusercontent.com/a/AATXAJwpLB...,"Friendly welcome, extra helpful. They gave de...",Audi Zentrum München Hochstraße,5,2 months ago,False,2022,False
1,Natalie Gentner,NaN,1,https://www.google.com/maps/contrib/1139429743...,https://lh3.googleusercontent.com/a/AATXAJxur1...,Excellent service and an overall great experie...,Audi Zentrum München Hochstraße,5,a year ago,False,2021,False
2,Luvje HiLuX,NaN,2,https://www.google.com/maps/contrib/1103578390...,https://lh3.googleusercontent.com/a-/AOh14Ggdn...,Very friendly employees and very good service. 👍,Audi Zentrum München Hochstraße,5,a week ago,False,2022,True
3,Aron Borbath,Local Guide,0,https://www.google.com/maps/contrib/1170914828...,https://lh3.googleusercontent.com/a/AATXAJwepl...,"Looks amazing, very kind service but very expe...",Audi Zentrum München Hochstraße,4,2 weeks ago,False,2022,False
4,George A. Zombanakis,NaN,1,https://www.google.com/maps/contrib/1164324891...,https://lh3.googleusercontent.com/a-/AOh14Gjpg...,I have arrived in Munich travelling to Greece ...,Audi Zentrum München Hochstraße,5,2 years ago,False,2020,True
...,...,...,...,...,...,...,...,...,...,...,...,...
6742,Wolfgang Brocksch,Local Guide,0,https://www.google.com/maps/contrib/1170636426...,https://lh3.googleusercontent.com/a/AATXAJyXlO...,Good service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,3 years ago,True,2019,False
6743,Capricorn_1957 “Capricorn_1957”,Local Guide,0,https://www.google.com/maps/contrib/1047367352...,https://lh3.googleusercontent.com/a-/AOh14Gjk6...,A city 👍👍👍,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True
6744,Alexandra Graupner (Xandi),NaN,67,https://www.google.com/maps/contrib/1139925999...,https://lh3.googleusercontent.com/a/AATXAJzpW2...,Super Service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,3 years ago,True,2019,False
6745,Yakamoz xxxx,NaN,16,https://www.google.com/maps/contrib/1065123727...,https://lh3.googleusercontent.com/a-/AOh14Gjhk...,VW THE CAR ...,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True


In [3]:
# get the BMW branches name in Munich from dataset
munich_list = [x for x in df['Branch'].unique() if ('München' in x)]
munich_list

['Audi Zentrum München Hochstraße',
 'Audi Zentrum München Albrechtstraße',
 'Audi Trudering München',
 'Audi München Starnberg',
 'Audi plus Zentrum München']

In [4]:
# create new dataset with the BMW branches in Munich
df_munich = df[df['Branch'].apply(lambda x: True if x in munich_list else False)].copy()
df_munich.reset_index(drop = True, inplace = True)
df_munich

,Username,Description,Number of reviews,Profile URL,Image URL,Review,Branch,Rating,Date,Translated,Year,Profile picture
0,Hayrullah AYAN,NaN,2,https://www.google.com/maps/contrib/1009337064...,https://lh3.googleusercontent.com/a/AATXAJwpLB...,"Friendly welcome, extra helpful. They gave de...",Audi Zentrum München Hochstraße,5,2 months ago,False,2022,False
1,Natalie Gentner,NaN,1,https://www.google.com/maps/contrib/1139429743...,https://lh3.googleusercontent.com/a/AATXAJxur1...,Excellent service and an overall great experie...,Audi Zentrum München Hochstraße,5,a year ago,False,2021,False
2,Luvje HiLuX,NaN,2,https://www.google.com/maps/contrib/1103578390...,https://lh3.googleusercontent.com/a-/AOh14Ggdn...,Very friendly employees and very good service. 👍,Audi Zentrum München Hochstraße,5,a week ago,False,2022,True
3,Aron Borbath,Local Guide,0,https://www.google.com/maps/contrib/1170914828...,https://lh3.googleusercontent.com/a/AATXAJwepl...,"Looks amazing, very kind service but very expe...",Audi Zentrum München Hochstraße,4,2 weeks ago,False,2022,False
4,George A. Zombanakis,NaN,1,https://www.google.com/maps/contrib/1164324891...,https://lh3.googleusercontent.com/a-/AOh14Gjpg...,I have arrived in Munich travelling to Greece ...,Audi Zentrum München Hochstraße,5,2 years ago,False,2020,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1260,rosalba scarpino,NaN,190,https://www.google.com/maps/contrib/1122730340...,https://lh3.googleusercontent.com/a-/AOh14GiL6...,Interesting....,Audi plus Zentrum München,5,3 years ago,True,2019,True
1261,P.-Matthieu Fritsch,Local Guide,0,https://www.google.com/maps/contrib/1079194976...,https://lh3.googleusercontent.com/a-/AOh14Ggnm...,just a dream!,Audi plus Zentrum München,5,6 years ago,True,2016,True
1262,Gianfranco Montagna,NaN,2,https://www.google.com/maps/contrib/1007885446...,https://lh3.googleusercontent.com/a/AATXAJwEC8...,Very beautifull!!!!,Audi plus Zentrum München,5,2 years ago,True,2020,False
1263,Ludwig Albrecht,Local Guide,0,https://www.google.com/maps/contrib/1107409691...,https://lh3.googleusercontent.com/a-/AOh14Gisz...,Perfect!,Audi plus Zentrum München,5,6 years ago,True,2016,True


# DATA CLEANING

### If we want we can analyze only the branches in Munich, but i would like to analyse all the reviews

In [3]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [5]:
# Let's take a look at the updated text
df["Review"] = df["Review"].apply(round1)
df

,Username,Description,Number of reviews,Profile URL,Image URL,Review,Branch,Rating,Date,Translated,Year,Profile picture
0,Hayrullah AYAN,NaN,2,https://www.google.com/maps/contrib/1009337064...,https://lh3.googleusercontent.com/a/AATXAJwpLB...,friendly welcome extra helpful they gave deta...,Audi Zentrum München Hochstraße,5,2 months ago,False,2022,False
1,Natalie Gentner,NaN,1,https://www.google.com/maps/contrib/1139429743...,https://lh3.googleusercontent.com/a/AATXAJxur1...,excellent service and an overall great experie...,Audi Zentrum München Hochstraße,5,a year ago,False,2021,False
2,Luvje HiLuX,NaN,2,https://www.google.com/maps/contrib/1103578390...,https://lh3.googleusercontent.com/a-/AOh14Ggdn...,very friendly employees and very good service 👍,Audi Zentrum München Hochstraße,5,a week ago,False,2022,True
3,Aron Borbath,Local Guide,0,https://www.google.com/maps/contrib/1170914828...,https://lh3.googleusercontent.com/a/AATXAJwepl...,looks amazing very kind service but very expen...,Audi Zentrum München Hochstraße,4,2 weeks ago,False,2022,False
4,George A. Zombanakis,NaN,1,https://www.google.com/maps/contrib/1164324891...,https://lh3.googleusercontent.com/a-/AOh14Gjpg...,i have arrived in munich travelling to greece ...,Audi Zentrum München Hochstraße,5,2 years ago,False,2020,True
...,...,...,...,...,...,...,...,...,...,...,...,...
6742,Wolfgang Brocksch,Local Guide,0,https://www.google.com/maps/contrib/1170636426...,https://lh3.googleusercontent.com/a/AATXAJyXlO...,good service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,3 years ago,True,2019,False
6743,Capricorn_1957 “Capricorn_1957”,Local Guide,0,https://www.google.com/maps/contrib/1047367352...,https://lh3.googleusercontent.com/a-/AOh14Gjk6...,a city 👍👍👍,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True
6744,Alexandra Graupner (Xandi),NaN,67,https://www.google.com/maps/contrib/1139925999...,https://lh3.googleusercontent.com/a/AATXAJzpW2...,super service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,3 years ago,True,2019,False
6745,Yakamoz xxxx,NaN,16,https://www.google.com/maps/contrib/1065123727...,https://lh3.googleusercontent.com/a-/AOh14Gjhk...,vw the car,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True


In [6]:
def lan_detect(text):
    try:
        new_text = detect(text)
    except:
        new_text = None
    return new_text

In [7]:
df["language"] = df["Review"].apply(lambda x: lan_detect(x))

#remove all non english rows
df = df[df["language"] == "en"]
df

,Username,Description,Number of reviews,Profile URL,Image URL,Review,Branch,Rating,Date,Translated,Year,Profile picture,language
0,Hayrullah AYAN,NaN,2,https://www.google.com/maps/contrib/1009337064...,https://lh3.googleusercontent.com/a/AATXAJwpLB...,friendly welcome extra helpful they gave deta...,Audi Zentrum München Hochstraße,5,2 months ago,False,2022,False,en
1,Natalie Gentner,NaN,1,https://www.google.com/maps/contrib/1139429743...,https://lh3.googleusercontent.com/a/AATXAJxur1...,excellent service and an overall great experie...,Audi Zentrum München Hochstraße,5,a year ago,False,2021,False,en
2,Luvje HiLuX,NaN,2,https://www.google.com/maps/contrib/1103578390...,https://lh3.googleusercontent.com/a-/AOh14Ggdn...,very friendly employees and very good service 👍,Audi Zentrum München Hochstraße,5,a week ago,False,2022,True,en
3,Aron Borbath,Local Guide,0,https://www.google.com/maps/contrib/1170914828...,https://lh3.googleusercontent.com/a/AATXAJwepl...,looks amazing very kind service but very expen...,Audi Zentrum München Hochstraße,4,2 weeks ago,False,2022,False,en
4,George A. Zombanakis,NaN,1,https://www.google.com/maps/contrib/1164324891...,https://lh3.googleusercontent.com/a-/AOh14Gjpg...,i have arrived in munich travelling to greece ...,Audi Zentrum München Hochstraße,5,2 years ago,False,2020,True,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6740,Rolf Leichsenring,Local Guide,0,https://www.google.com/maps/contrib/1184325580...,https://lh3.googleusercontent.com/a-/AOh14GgBy...,good customer service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,4,3 years ago,True,2019,True,en
6741,Werner Schuster,Local Guide,0,https://www.google.com/maps/contrib/1119926693...,https://lh3.googleusercontent.com/a-/AOh14Gh3I...,everything great,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,a year ago,True,2021,True,en
6743,Capricorn_1957 “Capricorn_1957”,Local Guide,0,https://www.google.com/maps/contrib/1047367352...,https://lh3.googleusercontent.com/a-/AOh14Gjk6...,a city 👍👍👍,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True,en
6745,Yakamoz xxxx,NaN,16,https://www.google.com/maps/contrib/1065123727...,https://lh3.googleusercontent.com/a-/AOh14Gjhk...,vw the car,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True,en


In [8]:
df.drop(["language"], axis=1, inplace=True)
df

,Username,Description,Number of reviews,Profile URL,Image URL,Review,Branch,Rating,Date,Translated,Year,Profile picture
0,Hayrullah AYAN,NaN,2,https://www.google.com/maps/contrib/1009337064...,https://lh3.googleusercontent.com/a/AATXAJwpLB...,friendly welcome extra helpful they gave deta...,Audi Zentrum München Hochstraße,5,2 months ago,False,2022,False
1,Natalie Gentner,NaN,1,https://www.google.com/maps/contrib/1139429743...,https://lh3.googleusercontent.com/a/AATXAJxur1...,excellent service and an overall great experie...,Audi Zentrum München Hochstraße,5,a year ago,False,2021,False
2,Luvje HiLuX,NaN,2,https://www.google.com/maps/contrib/1103578390...,https://lh3.googleusercontent.com/a-/AOh14Ggdn...,very friendly employees and very good service 👍,Audi Zentrum München Hochstraße,5,a week ago,False,2022,True
3,Aron Borbath,Local Guide,0,https://www.google.com/maps/contrib/1170914828...,https://lh3.googleusercontent.com/a/AATXAJwepl...,looks amazing very kind service but very expen...,Audi Zentrum München Hochstraße,4,2 weeks ago,False,2022,False
4,George A. Zombanakis,NaN,1,https://www.google.com/maps/contrib/1164324891...,https://lh3.googleusercontent.com/a-/AOh14Gjpg...,i have arrived in munich travelling to greece ...,Audi Zentrum München Hochstraße,5,2 years ago,False,2020,True
...,...,...,...,...,...,...,...,...,...,...,...,...
6740,Rolf Leichsenring,Local Guide,0,https://www.google.com/maps/contrib/1184325580...,https://lh3.googleusercontent.com/a-/AOh14GgBy...,good customer service,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,4,3 years ago,True,2019,True
6741,Werner Schuster,Local Guide,0,https://www.google.com/maps/contrib/1119926693...,https://lh3.googleusercontent.com/a-/AOh14Gh3I...,everything great,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,a year ago,True,2021,True
6743,Capricorn_1957 “Capricorn_1957”,Local Guide,0,https://www.google.com/maps/contrib/1047367352...,https://lh3.googleusercontent.com/a-/AOh14Gjk6...,a city 👍👍👍,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True
6745,Yakamoz xxxx,NaN,16,https://www.google.com/maps/contrib/1065123727...,https://lh3.googleusercontent.com/a-/AOh14Gjhk...,vw the car,Audi Zentrum Hof - Motor-Nützel Vertriebs-GmbH,5,2 years ago,True,2020,True


# NLP ENGINE

In [9]:
# Initialize the spaCy engine for English language
nlp_eng = spacy.load("en_core_web_sm")

In [10]:
# append all the english reviews to all_sent_eng() --> all sentences in english
# we will use all_sent_eng list to create a new dataframe with parsed english reviews
eng_sentences = []
for i, row in df.iterrows():
    review = row['Review']
    review = ' '.join(review.split())
    doc = nlp_eng(review)
    for sent in doc.sents:
        eng_sentences.append(sent.text)

In [11]:
eng_sentences

['friendly welcome extra helpful they gave detailed answers to all my questions',
 'excellent service and an overall great experience got an express service appointment on the next day very friendly and customer oriented service always reachable and very transparent with their work and costs great communication also with a third party that needed to be involved in my case recommendation',
 'very friendly employees and very good service 👍',
 'looks amazing very kind service but very expensive',
 'i have arrived in munich travelling to greece on august with a serious engine problem for my audi the personnel of the audi zentrum have been so efficient as to receive the car late that evening find the problem and repair it in a few hours to deliver it as good as new sometime at noon the next day special thanks to mr mateusz zmudatrzebiatowski for his help and understanding and congratulations to the entire zentrum management',
 'great servicepiker at the front desk arranged all and in min i 

In [12]:
# apply language detect methods after parsing the reviews
all_sent_eng = []
for sentence in eng_sentences:
    lang = lan_detect(sentence)
    if lang == "en":
        all_sent_eng.append(sentence)

In [13]:
len(all_sent_eng)

6519

# Combining similar words – lemmatization


In [14]:
# lemmatization process for all the parsed english reviews 
# lemma_sent_eng is for lemmatized parsed english reviews
lemma_sent_eng = []
for sent in all_sent_eng:
    doc = nlp_eng(sent)
    new_sent = " "
    new_sent = " ".join([token.lemma_ for token in doc])
    lemma_sent_eng.append(new_sent)

In [15]:
len(lemma_sent_eng)

6519

# Combine all parsed and lemmatized sentences¶

In [16]:
# creating english review dataframe with english review and english lemmatized review
# create dictionary 
d = {"eng_review":all_sent_eng, "lemma_eng_review":lemma_sent_eng}
# create dataset
english_rev= pd.DataFrame(d) 
english_rev

,eng_review,lemma_eng_review
0,friendly welcome extra helpful they gave detai...,friendly welcome extra helpful they give detai...
1,excellent service and an overall great experie...,excellent service and an overall great experie...
2,very friendly employees and very good service 👍,very friendly employee and very good service 👍
3,looks amazing very kind service but very expen...,look amazing very kind service but very expensive
4,i have arrived in munich travelling to greece ...,I have arrive in munich travel to greece on au...
...,...,...
6514,good customer service,good customer service
6515,everything great,everything great
6516,a city 👍👍👍,a city 👍 👍 👍
6517,vw the car,vw the car


# Removing Stopwords

In [17]:
#remove stop words for english language
def stop_eng(text):
    eng_stopwords = nlp_eng.Defaults.stop_words
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in eng_stopwords]
    sentence = " ".join(word for word in tokens_without_sw)
    return sentence

In [18]:
# create new column after removing stops words from eng review
english_rev["rev_stopped"] = english_rev["eng_review"].map(lambda x: stop_eng(x))

In [19]:
# create new column after removing stops words from lemmatized eng review
english_rev["lemma_stopped"] = english_rev["lemma_eng_review"].map(lambda x: stop_eng(x))

In [20]:
english_rev

,eng_review,lemma_eng_review,rev_stopped,lemma_stopped
0,friendly welcome extra helpful they gave detai...,friendly welcome extra helpful they give detai...,friendly welcome extra helpful gave detailed a...,friendly welcome extra helpful detailed answer...
1,excellent service and an overall great experie...,excellent service and an overall great experie...,excellent service overall great experience got...,excellent service overall great experience exp...
2,very friendly employees and very good service 👍,very friendly employee and very good service 👍,friendly employees good service 👍,friendly employee good service 👍
3,looks amazing very kind service but very expen...,look amazing very kind service but very expensive,looks amazing kind service expensive,look amazing kind service expensive
4,i have arrived in munich travelling to greece ...,I have arrive in munich travel to greece on au...,arrived munich travelling greece august engine...,I arrive munich travel greece august engine pr...
...,...,...,...,...
6514,good customer service,good customer service,good customer service,good customer service
6515,everything great,everything great,great,great
6516,a city 👍👍👍,a city 👍 👍 👍,city 👍👍👍,city 👍 👍 👍
6517,vw the car,vw the car,vw car,vw car


# Save the dataframe as csv

In [21]:
english_rev.to_csv("english.csv", index=False)
